In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io
import time
from base64 import b64encode
import cv2

def enhance_sharpness(image, amount=1.0):
    blurred = ndimage.gaussian_filter(image, sigma=1)
    mask = image - blurred
    sharpened = image + amount * mask
    return np.clip(sharpened, 0, 255)

def remove_noise_gaussian_blur(image):
    denoised_image = cv2.GaussianBlur(image, (7, 7), 0)
    return denoised_image

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False
)

processing_options = widgets.SelectMultiple(
    options=['Enhance Sharpness', 'Remove Noise'],
    value=[],
    description='Processing:'
)

process_button = widgets.Button(
    description='Process Image',
    button_style='success'
)

download_button = widgets.Button(
    description='Download Processed Image',
    button_style='info',
    disabled=True
)

output_image = widgets.Output()
metrics_output = widgets.Output()

def on_process_button_clicked(b):
    with output_image:
        clear_output()
        if not uploader.value:
            print("Please upload an image file.")
            return
        
        start_time = time.time()
        
        if isinstance(uploader.value, dict):
            uploaded_file = next(iter(uploader.value.values()))
        else:
            uploaded_file = uploader.value[0]
        content = uploaded_file['content']
        image = Image.open(io.BytesIO(content)).convert('RGB')
        image_np = np.array(image).astype(np.float64)
        
        if image_np.nbytes > 2 * 1024 * 1024 * 1024:
            print("Image is too large to process.")
            return
        
        original_image_np = image_np.copy()
        
        if 'Enhance Sharpness' in processing_options.value:
            image_np = enhance_sharpness(image_np)
        if 'Remove Noise' in processing_options.value:
            image_np = remove_noise_gaussian_blur(image_np)
        
        processing_time = time.time() - start_time
        if processing_time > 5:
            print("Processing took too long (>5 seconds).")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(original_image_np.astype(np.uint8))
        axes[0].set_title('Original Image')
        axes[0].axis('off')

        axes[1].imshow(image_np.astype(np.uint8))
        axes[1].set_title('Processed Image')
        axes[1].axis('off')
        
        plt.show()
        
        download_button.disabled = False

        processed_image_pil = Image.fromarray(np.uint8(image_np))
        download_button.processed_image_pil = processed_image_pil
    
def on_download_button_clicked(b):
    processed_image_pil = b.processed_image_pil
    with io.BytesIO() as output:
        processed_image_pil.save(output, format="PNG")
        contents = output.getvalue()
        data_url = "data:image/png;base64," + b64encode(contents).decode()
        display(widgets.HTML(f'<a download="processed_image.png" href="{data_url}" target="_blank">Click here to download your image</a>'))
    
process_button.on_click(on_process_button_clicked)
download_button.on_click(on_download_button_clicked)

display(widgets.VBox([
    widgets.Label("Upload an image (JPEG or PNG):"),
    uploader,
    processing_options,
    process_button,
    output_image,
    metrics_output,
    download_button
]))


HTML(value='<a download="processed_image.png" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAARMAAAC3CAIA…